# Test develop connectome analyses.

We develop the analyses and the (BBP) adapter together.
In this notebook we will prototype the adapter,
while developing the analyses in an editory.

In [9]:
import os
import numpy as np
import pandas as pd
from neuro_dmt.analysis.circuit.connectome import *
os.environ["LOGLEVEL"] = "PROD"

In [ ]:
from dmt.model.adapter import adapts
from dmt.model.interface import implements

@adapts()
class BlueBrainCircuitAdapter:

To test our code, and to explain our analysis, we will construct a simple example. 
The analysis code in ```PathwayConnectionProbabilityAnalysis``` expects a dataframe described in the test below:

In [ ]:
def test_measurement_conn_prob(dataframe):
    """
    Is dataframe good as connection probability measurement?
    """
    assert isinstance(dataframe.index, pd.MultiIndex),\
        """
        Connection Probability measurement dataframe should have a 
        MultiIndex. We got: {}
        """.format(dataframe.index)
    assert ("mean" in dataframe.columns or 
            "connection_probability" in dataframe.columns or
            "connprob" in dataframe.columns)
            
    

In [ ]:
df = pd.DataFrame([
    {"pre_mtype": "L1_BC",
     "post_mtype": "L1_SBC",
     "mean": 0.01},
    {"pre_mtype": "L1_BC",
     "post_mtype": "L1_PC",
     "mean": 0.03},
    {"pre_mtype": "L1_PC",
     "post_mtype": "L1_PC",
     "mean": 0.1}])\
    .set_index(["pre_mtype", "post_mtype"])

In [ ]:
df

In [ ]:
df.index.values

## Test circuit: Cell paper circuit

We have generated an instance of the cell paper circuit, that we will use to
test our connectome analyses.

In [2]:
from bluepysnap.circuit import Circuit
project =\
    os.path.join(
        "/gpfs", "bbp.cscs.ch", "project")
proj82 =\
    os.path.join(
        project, "proj82")
cell_paper_circuit_config =\
    os.path.join(
        proj82, "issues", "BBPP82-78", "CircuitConfig")                 
cell_paper_circuit = Circuit(cell_paper_circuit_config)

In [4]:
cells = cell_paper_circuit.cells.get()

In [5]:
cells.groupby("layer").agg("size")

layer
1     2374
2    23881
3    28748
4    32641
5    42771
6    89007
dtype: int64

In [7]:
conn = cell_paper_circuit.connectome

In [8]:
conn.afferent_gids(1)

array([   128,    300,   1424,   2033,   2078,   2504,   3747,   6771,
         7386,  24836,  65203,  65240,  67453,  96061, 105957], dtype=uint64)

Analysis will be easier to develop. We will convert a dataframe of the form 
discussed above to a matrix and then to a heat-map. The harder part is to 
develop the analysis. For the analysis we will need to compute probability that 
two neurons of given mtypes (pre --> post) are connected. This probability will 
depend on distance. So we can express the probability as
$$
Prob[X \sim Y \ |
\ mtype_X == mtype_{pre} \  \land
\  mtype_Y == mtype_{post} \  \land
\  distance(X, Y) <= D]
$$

## Mock Data
For the purposes of analysis, we may as well consider that the circuit is two 
data-frames: cell, and connectome data-frames. After all, except for a single 
exception, all our queries on the circuit will return either an array, or a 
pandas data-frame. One whole cell or connectome data-frame will contain several 
columns.

Let us consider the data we need to proceed with our analysis. The circuit 
provides cell gids, and each's cell's properties. Cell properties relevant to
our analysis are a cell's
1. MTYPE
2. Positions X, Y, Z
From the circuit connectome, for each cell (gid) we can retreive its afferent or
efferent cell (gids). For distance dependent connection probability that should be 
enough.

We will use a subset of mtypes.

In [ ]:
mtypes_test =[
    'L1_NGC-SA'
    'L1_SAC'
    'L23_NBC'
    'L2_TPC:A'
    'L4_NBC'
    'L4_SSC'
    'L4_UPC'
    'L5_TPC:B'
    'L5_TPC:C'
    'L6_CHC'
    'L6_DBC'
    'L6_NGC'
    'L6_SBC'
    'L6_TPC:A'
    'L6_TPC:C'
    'L6_UPC']

For cell positions, let us model the brain circuit as a cuboid --- the simplest 
geometry for computation.

In [ ]:
from dmt.tk.field import Field, WithFields


class MockCells(WithFields):

class MockCircuit(WithFields):
    """
    A cortical column in the shape of a cuboid. 
    The layers are stacked along the Y-axis.
    """
    layer_thicknesses = Field(
        """
        A dict mapping layer index to its thickness.
        Thickness will be measured in micro-meters.
        """)
    column_width = Field(
        """
        Length of the side of the base of the cortical column.
        The base will be a square.
        """)
    
    def __init__(self, *args, **kwargs):
        """
        Initialize required attributes to None, 
        and lazy load them.
        """
        self._cells = None
        self._connectome = None
        
    @property
    def cells()
    

# Playground
Where we play around with ideas.

In [ ]:
from collections import OrderedDict
class OrderedFieldMeta(type):
    """
    A class whose attributes will be ordered
    """
    @classmethod
    def __prepare__(mcs, name, bases):
        """
        Prepare to build a class.
        """
        print("prepare to build {} with {}".format(name, mcs))
        return OrderedDict()
    
    def __new__(mcs, name, bases, namespace):
        """
        Construct a class.
        """
        print("Construct a class {} with attributes in {}"\
              .format(name, namespace.__class__.__name__))
        print("It will have attributes: {}"\
              .format(namespace))
        return super().__new__(mcs, name, bases, namespace)

In [ ]:
from dmt.tk.field import Field, WithFields
class A(metaclass=OrderedFieldMeta):
    """
    A class with ordered fields
    """
    a = 1
    b = 2
    c = 3

In [ ]:
A.a

In [8]:
from neuro_dmt.models.sonata.circuit.mock import test_develop

In [9]:
df = test_develop.circuit_composition

In [11]:
df.cell_density.xs(5, level="layer")

,error,mean
mtype,,
L1_DAC,0.0,0.000000
L1_DLAC,0.0,0.000000
L1_HAC,0.0,0.000000
L1_NGC-DA,0.0,0.000000
L1_NGC-SA,0.0,0.000000
L1_SLAC,0.0,0.000000
L23_BP,0.0,0.000000
L23_BTC,0.0,0.000000
L23_ChC,0.0,0.000000


In [9]:
df.xs(2, level="layer").sum()

error         0.0
mean     131561.0
dtype: float64

In [12]:
sum(n % 2 == 0 for n in range(4))

2

In [11]:
def ei_ratio(i, t):
    i = (1. * i) / t
    return (1. - i) / i

In [12]:
print(ei_ratio(22., 78.))
print(ei_ratio(38, 252))
print(ei_ratio(20, 211))
print(ei_ratio(45, 267))
print(ei_ratio(37, 388))

2.5454545454545454
5.631578947368421
9.55
4.933333333333334
9.486486486486488


In [77]:
import yaml
rat_data_file =\
    os.path.join(
    "/Users/vishalsoodmuchu/work/bbp/work/validations/",
    "dmt/examples/datasets/cortex/sscx/rat/composition/",
    "cell_density/DeFelipe2017.yaml")
with open(rat_data_file, "r") as f:
    rat_dicts = yaml.load(f)["circuits"]
    rats = pd.concat(
        pd.DataFrame({
            "rat": 6 *[rat],
             "layer": range(1,7),
            "density": d["densities"],
            "thickness": d["thicknesses"]})
            for rat, d in rat_dicts.items())\
           .set_index(["rat", "layer"])

In [81]:
rats.groupby("layer").agg(["mean", "std", "size"])

density                      thickness                
                mean           std size        mean        std size
layer                                                              
1       13195.833333   4294.060755    6  144.700000  12.469643    6
2      157413.666667  18643.104234    6  130.566667  12.109445    6
3       77739.333333   2728.654736    6  309.633333  13.065323    6
4      164617.333333  11849.600916    6  166.300000   6.904491    6
5       77869.833333   8887.210483    6  460.666667  31.706319    6
6      121972.500000  14748.034116    6  614.416667  46.233642    6

In [32]:
[pd.DataFrame(d) for rat, d in rats.items()]

[   densities  thicknesses
 0      17218        122.3
 1     122006        113.5
 2      75306        302.9
 3     169828        176.4
 4      90956        477.9
 5     108090        647.3,    densities thicknesses
 0       7040       148.5
 1     169299       121.7
 2      81310       303.7
 3     172339       159.2
 4      71296      412.,8
 5     111238       627.1,    densities  thicknesses
 0      15337        160.5
 1     171045        126.4
 2      74956        303.8
 3     150973        161.2
 4      80783        442.6
 5     135638        536.6,    densities  thicknesses
 0      17290        143.2
 1     162856        135.4
 2      78259        296.5
 3     178580        160.9
 4      84361        447.7
 5     134840        603.5,    densities  thicknesses
 0      13190        146.6
 1     151972        144.5
 2      80528        319.7
 3     166547        168.9
 4      71168        486.9
 5     135599        670.2,    densities  thicknesses
 0       9100        147.1
 1     1